In [3]:
import heapq, random

class PriorityQueue:
    """
      Implements a priority queue data structure. Each inserted item
      has a priority associated with it and the client is usually interested
      in quick retrieval of the lowest-priority item in the queue. This
      data structure allows O(1) access to the lowest-priority item.
    """
    def  __init__(self):
        self.heap = []
        self.count = 0

    def push(self, item, priority):
        entry = (priority, self.count, item)
        heapq.heappush(self.heap, entry)
        self.count += 1

    def pop(self):
        (_, _, item) = heapq.heappop(self.heap)
        return item

    def isEmpty(self):
        return len(self.heap) == 0

    def update(self, item, priority):
        # If item already in priority queue with higher priority, update its priority and rebuild the heap.
        # If item already in priority queue with equal or lower priority, do nothing.
        # If item not in priority queue, do the same thing as self.push.
        for index, (p, c, i) in enumerate(self.heap):
            if i == item:
                if p <= priority:
                    break
                del self.heap[index]
                self.heap.append((priority, c, item))
                heapq.heapify(self.heap)
                break
        else:
            self.push(item, priority)

In [4]:
class Person:
    def __init__(self, height: int) -> None:
        """
        height: height in inches rounded to nearest whole number
        """
        self.height = height
        self.wingspan = height * 1.06
        self.reach = height * 1.35
        self.leg_length = height * 0.5

In [5]:
class Hold:
    def __init__(self, x: int, y: int, diff: float, width: float, height: float, angle: int):
        #Coords = top left corner
        self.x = x
        self.y = y
        self.diff = diff
        self.width = width
        self.height = height
        self.angle = angle
    def getCenter(self):
        return (self.coords[0] + self.width/2, self.coords[1] + self.height/2)
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    def __gt__(self, other):
        return self.y > other.y
    def __lt__(self, other):
        return self.y < other.y
    def __ge__(self, other):
        return self.y >= other.y
    def __le__(self, other):
        return self.y <= other.y
    
    def __repr__(self):
        return (f"""Hold: Top left at {self.x}, {self.y}
                Width = {self.width}, Height = {self.height}
                Difficulty = {round(self.diff, 2)}/10, Angle = {self.angle} degrees\n""")

In [6]:
from enum import Enum

class LimbName(Enum):
    LEFT_HAND = "Left Hand"
    RIGHT_HAND = "Right Hand"
    LEFT_LEG = "Left Leg"
    RIGHT_LEG = "Right Leg"


In [7]:
class Limb:
    def __init__(self, name: LimbName, strength: int, flexibility: int, hold: Hold):
        self.name, self.strength, self.flexibility, self.hold = name, strength, flexibility, hold
    def __repr__(self):
        return (f"{self.name} at {self.hold}")

In [8]:
class Route:
    def __init__(self, holds: Hold, start1: Hold, start2: Hold, finish: Hold):
        self.holds = holds
        self.start_hold1 = start1
        self.start_hold2 = start2
        self.finish_hold = finish

In [15]:
import math
from copy import copy

class State:
    def __init__(self, lf: Limb, rf: Limb, lh: Limb, rh: Limb, person: Person, route: Route):
        self.lf, self.rf, self.lh, self.rh = lf, rf, lh, rh
        self.person = person
        self.moves = []
        self.costs = []
        self.route = route
        self.wall_height_inches = 180 # Represents real height of wall <- need to measure
        self.wall_height_pixels = 3321 #placeholder <- Need to extract from image

    # Not sure where to put this
    def inches_to_pixels(self, inches: int):
        return (self.wall_height_pixels / self.wall_height_inches) * inches

    def __eq__(self, other):
        return ((self.lf.hold == other.lf.hold and self.rf.hold == other.rf.hold 
                and self.rh.hold == other.rh.hold and self.lh.hold == other.lh.hold)
                or (self.lf.hold == other.rf.hold and self.rf.hold == other.lf.hold 
                and self.rh.hold == other.lh.hold and self.lh.hold == other.rh.hold))

    def __repr__(self):
        return f"{self.moves}"

    def getStateSuccessors(self):
        succs = []
        limbs = [self.lf, self.rf, self.lh, self.rh]
        for i in range(len(limbs)):
            neighs = self.getNeighbors(limbs[i])
            for neigh in neighs:
                #print("Hi neighbor")
                new_state = copy(self)
                if i == 0:
                    new_state.lf = Limb(LimbName.LEFT_LEG, 2.5, 8, neigh)
                    action = (self.lf, neigh)
                if i == 1:
                    new_state.rf = Limb(LimbName.RIGHT_LEG, 2.5, 8, neigh)
                    action = (self.rf, neigh)
                if i == 2:
                    new_state.lh = Limb(LimbName.LEFT_HAND, 8, 2, neigh)
                    action = (self.lh, neigh)
                if i == 3:
                    new_state.rh = Limb(LimbName.RIGHT_HAND, 8, 2, neigh)
                    action = (self.rh, neigh)
                succs.append((new_state, action))
        #print(succs)
        return succs

    def getNeighbors(self, limb):
        neighbors = []
        for hold in self.route.holds:
            if not hold == limb.hold:
                if limb.name in [LimbName.LEFT_LEG, LimbName.RIGHT_LEG]:
                    #print("Checking leg neighbors")
                    #print(f"Height diff: {abs(hold.y - limb.hold.y)}")
                    #print(f"Leg length: {self.inches_to_pixels(self.person.leg_length)}")
                    # low_arm is max because pixels go from top to bottom
                    low_arm = max([self.lh.hold.y, self.rh.hold.y])
                    if (0 < limb.hold.y - hold.y < self.inches_to_pixels(self.person.leg_length) and 
                        abs(hold.x - limb.hold.x) < self.inches_to_pixels(self.person.leg_length) and
                        hold.y > low_arm): 
                        neighbors.append(hold)
                elif limb.name in [LimbName.LEFT_HAND, LimbName.RIGHT_HAND]:
                    #print("Checking arm neighbors")
                    #print("HAND TIME 1")
                    upper_leg, lower_leg = sorted([self.lf.hold.y, self.rf.hold.y])
                    if (abs(hold.x - limb.hold.x) < self.inches_to_pixels(self.person.wingspan) and
                        lower_leg - hold.y < self.inches_to_pixels(self.person.height * 0.8) and hold.y < upper_leg and 0 < limb.hold.y - hold.y):
                        #print(hold.y - upper_leg)
                        #print("HAND TIME 2")
                        neighbors.append(hold)
        return neighbors
        
def moveDifficulty(state: State, limb: Limb, next_hold: Hold):
    distance = math.sqrt(((limb.hold.x - next_hold.x) ** 2) + ((limb.hold.y - next_hold.y) ** 2))
    distance_diff = distance

    new_state = State(copy(state.lf), copy(state.rf), copy(state.lh), copy(state.rh), state.person, state.route)
    new_state_limbs = [new_state.lh, new_state.rh, new_state.lf, new_state.rf]
    for new_state_limb in new_state_limbs:
        if new_state_limb.name == limb.name:
            new_state_limb.hold = None
    state_without_limb_difficulty = 0.3 * stateDifficulty(new_state)
    distance_diff *= 0.05
    move_diff = distance_diff + state_without_limb_difficulty
    return move_diff

def stateDifficulty(state: State):
    if state.lh.hold != None and state.rh.hold != None:
        average_hands_x = (state.lh.hold.x + state.rh.hold.x) / 2
        average_hands_y = (state.lh.hold.y + state.rh.hold.y) / 2
    else:
        if state.rh.hold == None:
            average_hands_x = state.lh.hold.x  
            average_hands_y = state.lh.hold.y
        else:
            average_hands_y = state.rh.hold.y
            average_hands_x = state.rh.hold.x
    if state.lf.hold != None and state.rf.hold != None:
        average_legs_x = (state.lf.hold.x + state.rf.hold.x) / 2
        average_legs_y = (state.lf.hold.y + state.rf.hold.y) / 2 
    else:
        if state.rf.hold == None:
            average_legs_x = state.lf.hold.x
            average_legs_y = state.lf.hold.y
        else:
            average_legs_y = state.rf.hold.y
            average_legs_x = state.rf.hold.x

    hands_difference_x = abs(state.rh.hold.x - state.lh.hold.x)if state.lh.hold != None and state.rh.hold != None else 0
    hands_difference_y = abs(state.rh.hold.y - state.lh.hold.y) if state.lh.hold != None and state.rh.hold != None else 0

    legs_difference_x = abs(state.rf.hold.x - state.lf.hold.x) if state.lf.hold != None and state.rf.hold != None else 0
    legs_difference_y = abs(state.lf.hold.y - state.rf.hold.y) if state.lf.hold != None and state.rf.hold != None else 0

    hands_difference_raw_x = state.rh.hold.x - state.lh.hold.x if state.lh.hold != None and state.rh.hold != None else 0
    legs_difference_raw_x = state.rf.hold.x - state.lf.hold.x if state.lf.hold != None and state.rf.hold != None else 0

    leg_match_diff = 0
    if state.lf.hold != None and state.rf.hold != None and state.lf.hold.x - state.rf.hold.x == 0:
        leg_match_diff = 25

    cross_diff = 0
    if hands_difference_raw_x < 0:
        cross_diff += 2 * abs(hands_difference_raw_x / 88)
    if legs_difference_raw_x < 0:
        cross_diff += 50
    if hands_difference_raw_x < 0 and legs_difference_raw_x < 0:
        cross_diff *= 3

    diff = 0
    center_diff = abs(average_hands_x - average_legs_x)

    target_distance = state.inches_to_pixels(state.person.height * 0.8)
    distance_diff = abs(average_legs_y - average_hands_y) - target_distance

    raw_difficulty_score = (distance_diff ** 2)

    limb_strength_diff = 0
    angle_diff = 0
    for limb in [state.lh, state.rh, state.lf, state.rf]:
        if limb.hold != None:
            limb_strength_diff += limb.hold.diff / limb.strength
            if limb.name == LimbName.LEFT_HAND:
                if 315 >= limb.hold.angle >= 270:
                    angle_diff += 2
                elif 90 >= limb.hold.angle or limb.hold.angle > 315:
                    angle_diff += 1
                elif 180 >= limb.hold.angle > 90:
                    angle_diff += 2.5
                else:
                    angle_diff += 3
            if limb.name == LimbName.RIGHT_HAND:
                if 90 >= limb.hold.angle >= 45:
                    angle_diff += 2
                elif 45 >= limb.hold.angle or limb.hold.angle > 270:
                    angle_diff += 1
                elif 270 >= limb.hold.angle > 180:
                    angle_diff += 2.5
                else:
                    angle_diff += 3
            if limb.name in [LimbName.LEFT_LEG, LimbName.RIGHT_LEG]:
                if 90 <= limb.hold.angle <= 270:
                    angle_diff += 2
        else:
            limb_strength_diff += 6
    separation_diff = 0
    separation_diff += 0.1 * hands_difference_y
    separation_diff += 0.1 * legs_difference_y



    if hands_difference_x > state.inches_to_pixels(0.8 * state.person.wingspan):
        separation_diff += 0.5 * hands_difference_x
    separation_diff += 0.5 * legs_difference_y
    if legs_difference_x > state.inches_to_pixels(0.6 * state.person.wingspan):
        separation_diff += 0.5 * legs_difference_x
    center_diff *= .04
    raw_difficulty_score *= 0.0005
    angle_diff *= 1
    limb_strength_diff *= 4
    separation_diff *= 0.15
    cross_diff *= 1
    leg_match_diff *= 1
    diff += center_diff + raw_difficulty_score + angle_diff + limb_strength_diff + separation_diff + leg_match_diff + cross_diff
    print(f"center = {center_diff}, scaled = {raw_difficulty_score}, angle = {angle_diff}, strength = {limb_strength_diff}, cross = {cross_diff}")
    return diff

In [10]:
import numpy as np
from copy import copy


class RouteFinder:
    """
    Class to find the route of a wall
    Takes in a state
    """
    WALL_HEIGHT = 180
    def __init__(self, state):
        self.state = state
        self.reach = state.person.reach
      
    # Helper function used to square the costs in the list of costs
    def getCostValue(self, costs):
        return sum([cost ** 2 for cost in costs])
        
    def uniformCostSearch(self):
        explored = []
        frontier = PriorityQueue()
        num = 0
        frontier.push(self.state, 0)
        while not frontier.isEmpty():
            cur_state = frontier.pop()
            explored.append(cur_state)
            if cur_state.lh.hold == self.state.route.finish_hold and cur_state.rh.hold == self.state.route.finish_hold:
                print("TAAAAAAAAAAAAAAKE")
                print(cur_state.costs)
                print(self.getCostValue(cur_state.costs))
                return cur_state.moves
            for next_state, action in cur_state.getStateSuccessors():
                if next_state not in explored: 
                    num += 1
                    if num % 500 == 0:
                        print(f"Checking state #{num} with move length {len(cur_state.moves)}")
                    next_state.costs = copy(cur_state.costs)
                    next_state.costs.append(stateDifficulty(next_state) + moveDifficulty(cur_state, action[0], action[1]))
                    next_state.moves = copy(cur_state.moves)
                    next_state.moves.append(action)
                    if next_state in [tup[2] for tup in frontier.heap]:
                        frontier.update(next_state, self.getCostValue(next_state.costs))
                    else:
                        frontier.push(next_state, self.getCostValue(next_state.costs))
        return []


In [ ]:
from get_holds import get_holds_array

holds = get_holds_array('../images//walls/redroute.jpg', [0, 25, 0], 10)

holds.sort()

In [12]:
def moveToText(action, route):
    output = "- Move your "
    if action[0].name == LimbName.LEFT_LEG:
        output += "left leg "
    elif action[0].name == LimbName.LEFT_HAND:
        output += "left hand "
    elif action[0].name == LimbName.RIGHT_LEG:
        output += "right leg "
    elif action[0].name == LimbName.RIGHT_HAND:
        output += "right hand "
    output += "to hold number "
    output += str(route.holds.index(action[1]))
    output += " from the top"
    return output

In [17]:
route = Route(holds = holds, start1 = holds[10], start2 = holds[8], finish = holds[0])
print(holds[12].x, holds[11].x)
lh = Limb(LimbName.LEFT_HAND, 2.5, 8, route.start_hold1)
rh = Limb(LimbName.RIGHT_HAND, 2.5, 8, route.start_hold2)
lf = Limb(LimbName.LEFT_LEG, 8, 5, route.holds[15])
rf = Limb(LimbName.RIGHT_LEG, 8, 5, route.holds[15])
ondra = Person(70)
liam = Person(68)
rachel = Person(66)
anna = Person(63)
short = Person(59)
luisa = Person(64)
tall = Person(78)
state1 = State(lf, rf, lh, rh, rachel, route)
a_star1 = RouteFinder(state=state1)
results1 = a_star1.uniformCostSearch()

for action in results1:
    print(moveToText(action, route))

768.2842 1172.2876
center = 8.987470703125, scaled = 150.29269341098103, angle = 4.5, strength = 31.442523574829103, cross = 0
center = 12.525273437500001, scaled = 64.3924226034859, angle = 4.5, strength = 39.4425235748291, cross = 0
center = 0.9074023437500001, scaled = 140.59863080514344, angle = 4.5, strength = 26.620686721801757, cross = 0
center = 12.525273437500001, scaled = 64.3924226034859, angle = 4.5, strength = 39.4425235748291, cross = 0
center = 18.086103515625002, scaled = 124.24264967090043, angle = 4.5, strength = 30.484515762329103, cross = 50
center = 12.525273437500001, scaled = 64.3924226034859, angle = 4.5, strength = 39.4425235748291, cross = 0
center = 8.852353515625, scaled = 102.7213621332589, angle = 6.5, strength = 30.24428596496582, cross = 0
center = 12.525273437500001, scaled = 64.3924226034859, angle = 4.5, strength = 39.4425235748291, cross = 0
center = 8.987470703125, scaled = 150.29269341098103, angle = 4.5, strength = 31.442523574829103, cross = 50
c

In [14]:
print(len(holds))
for i in range(len(holds)):
    print(f"{i}: {holds[i].x}, {holds[i].y}, {holds[i].diff}")



17
0: 659.6800537109375, 582.14892578125, 3.851250648498535
1: 943.9332885742188, 808.7344970703125, 3.1152353286743164
2: 542.1054077148438, 884.8870239257812, 4.027359962463379
3: 999.0489501953125, 1112.322998046875, 5.899986267089844
4: 927.7785034179688, 1391.28466796875, 7.116968154907227
5: 1364.2099609375, 1457.0947265625, 3.569828510284424
6: 720.4488525390625, 1724.4197998046875, 5.3884596824646
7: 1451.65185546875, 1729.7177734375, 3.4041407108306885
8: 1345.602783203125, 2012.46728515625, 3.0565712451934814
9: 800.716796875, 2018.7711181640625, 6.548148155212402
10: 726.4889526367188, 2301.441162109375, 3.470005989074707
11: 1172.28759765625, 2393.466796875, 10.0
12: 768.2841796875, 2429.41943359375, 6.98635196685791
13: 1627.21923828125, 2493.01416015625, 9.4012451171875
14: 1165.5316162109375, 2583.46435546875, 9.25110149383545
15: 1349.177734375, 2772.248046875, 10.0
16: 1241.4046630859375, 3045.589599609375, 3.9042158126831055
